In [17]:
# Imported libraries
import pydicom
import matplotlib.pyplot as plt
import numpy as np
import sys
import matplotlib as mpl
import imageio
import cv2
import sys
import os
import pickle

In [18]:
# Place directory paths and declare needed
# variables for initiating the general categorization
# and storing of patient scans arrays

original_directory_path = '/media/omar/New Volume/TCIA Data/GBM/Data/TCGA-GBM/'
segmented_directory_path = '/home/omar/Desktop/DropboxData/Dropbox-GBM/TCGA-GBM/'
patients = []

# Get all the patient IDs that will be used for the training phase and sort them
for patient in os.listdir(segmented_directory_path):
    patients.append(patient)
patients.sort()
# We now have a list of all the patients needed for the training phase
# print (patients)

In [19]:
# Aiding modules and functions

# Module to handle the existence of subfolders for multiple patient visits
def multipleVisits(patient_id, segmented_scans):
#     Purpose and working principle:
#     Iterates over the original patient subfolders till it finds scan folders relevant by scan ID
#     Inputs are, patient ID and an array containing
#     the segmentation filenames in the form 'seg_corrected_AX-*****.dcm'
#     Returns the subfolders containing the relevant original scans (usually one folder)
    subfolders = os.listdir(original_directory_path+patient_id)
    segmented_ids = []
    for i in segmented_scans:
        segmented_ids.append(i[-9:-4])
    target_folders = []
    for folder in subfolders:
        images_folder = os.listdir(original_directory_path+patient_id+"/"+folder)
        for scans in images_folder:
            if scans[-5:] in segmented_ids and folder not in target_folders:
                target_folders.append(folder)
    return target_folders


In [21]:
# After having the training patients recognized, we now need to start
# creating pickle files for every patient to store the data at

# Pickle file Information
# Name: Patient
# Data included:
# a. original scan images
# b. arrays for each of the segmented scans


result_directory_path = '/home/omar/Desktop/TrainingResult/'
os.chdir(result_directory_path)

for patient_id in patients:
#     Getting the corrected segmented scan filenames for every patient (ID at indexing -9:-4)
    patient_segmented_files = []
    for scan in os.listdir(segmented_directory_path+patient_id+'/DSO/'):
        if "corrected" in scan:
            patient_segmented_files.append(scan)
    
#     Pickle file for patient initialized
    pickleFile = open("%s.pickle" % (patient_id) ,"wb")
#     Data Array for patient initialized
    data_array = []
#     Retreiving files of the original patient scans for designated scan
#     Problem occured, not all patient directories have single scan history
#     Patients with such issue are: 06-0138, 19-5960. Resolved by validating ID for every scan
#     Status: Resolved by defining an aiding module multipleVisits

# Firstly handling normal easy cases by the following if statement
# If there is only one subfolder in the original patient scan for one date only
    if(len(os.listdir(original_directory_path+patient_id)) == 1):
        visit_date = os.listdir(original_directory_path+patient_id)[0]
        for scan_directory in os.listdir(original_directory_path+patient_id+"/"+visit_date):
            if any(scan_directory[-5:] in segment_scan for segment_scan in patient_segmented_files):
                folder_containing_scans = scan_directory
                path_for_scans = original_directory_path+patient_id+"/"+visit_date+"/"+folder_containing_scans
                scan_array = []
                images = os.listdir(path_for_scans)
                for image in images:
                    ds = pydicom.dcmread(path_for_scans+"/"+image).pixel_array
                    scan_array.append(ds)
                data_array.append(scan_array)
        pickle.dump(data_array,pickleFile)
        pickleFile.close()
#     In case of having multiple visits by the patient
    else:
        folder_containing_scans = multipleVisits(patient_id, patient_segmented_files)
        for folder in folder_containing_scans:
            visit_date = folder
        for scan_directory in os.listdir(original_directory_path+patient_id+"/"+visit_date):
            if any(scan_directory[-5:] in segment_scan for segment_scan in patient_segmented_files):
                folder_containing_scans = scan_directory
                path_for_scans = original_directory_path+patient_id+"/"+visit_date+"/"+folder_containing_scans
                scan_array = []
                images = os.listdir(path_for_scans)
                for image in images:
                    ds = pydicom.dcmread(path_for_scans+"/"+image).pixel_array
                    scan_array.append(ds)
                data_array.append(scan_array)
        pickle.dump(data_array,pickleFile)
        pickleFile.close()                


In [22]:
# Completed so far, pickle for gathering all designated original data scans for every segmentation
# scan has been completed. Testing for results. Anomalies detected with multiple
# with extra number of scans
# Needed for completing the training of the task, adding the segmentation array with every patient's array

In [30]:
result_files = os.listdir(result_directory_path)
for file in result_files:
    patient = open("%s" % (result_directory_path+file), "rb")
    patient_data = pickle.load(patient)
    for i in patient_data:
        print (file, len(i))

TCGA-02-0006.pickle 23
TCGA-02-0006.pickle 23
TCGA-02-0006.pickle 23
TCGA-02-0006.pickle 23
TCGA-08-0355.pickle 44
TCGA-08-0355.pickle 31
TCGA-08-0355.pickle 108
TCGA-08-0355.pickle 108
TCGA-02-0085.pickle 22
TCGA-02-0085.pickle 22
TCGA-02-0085.pickle 22
TCGA-02-0085.pickle 22
TCGA-76-4932.pickle 20
TCGA-76-4932.pickle 40
TCGA-76-4932.pickle 19
TCGA-76-4932.pickle 20
TCGA-06-0138.pickle 69
TCGA-06-0138.pickle 36
TCGA-06-0138.pickle 72
TCGA-06-0138.pickle 36
TCGA-19-2624.pickle 25
TCGA-19-2624.pickle 192
TCGA-19-2624.pickle 25
TCGA-19-2624.pickle 25
TCGA-76-4935.pickle 32
TCGA-76-4935.pickle 30
TCGA-76-4935.pickle 28
TCGA-76-4935.pickle 32
TCGA-06-2570.pickle 67
TCGA-06-2570.pickle 34
TCGA-06-2570.pickle 67
TCGA-06-2570.pickle 34
TCGA-76-6280.pickle 28
TCGA-76-6280.pickle 48
TCGA-76-6280.pickle 25
TCGA-76-6280.pickle 25
TCGA-06-6389.pickle 45
TCGA-06-6389.pickle 78
TCGA-06-6389.pickle 39
TCGA-06-6389.pickle 39
TCGA-06-0149.pickle 36
TCGA-06-0149.pickle 70
TCGA-06-0149.pickle 36
TCGA-06-